File imports all of the features generated from the various other data sources  
input = all the pkl and parquet files from other data sources  
output = master.parquet file  
checks are place intermittantly throughtout and can be used to gauge status of dataframe  

In [3]:
# library imports
import pandas as pd

In [4]:
# skip if recreating, needed otherwise
df_master = pd.read_parquet('./master.parquet')

In [ ]:
# see all columns prior to building if imported
df_master.columns

In [2]:
# start build master dataframe
# read compiled ldap file to begin
df_ldap = pd.read_pickle('../ldap/ldap.pkl')

In [3]:
# check
df_ldap.head(5)

,employee_name,user_id,email,role,business_unit,functional_unit,department,team,supervisor,date
0,Macey Colleen Nash,MCN0973,Macey.Colleen.Nash@dtaa.com,ElectricalEngineer,1,3 - Manufacturing,2 - ManufacturingEngineering,NaN,Ann Hannah Dickerson,2009-12
1,Nelle Sharon Cherry,NSC0622,Nelle.Sharon.Cherry@dtaa.com,Technician,1,5 - SalesAndMarketing,3 - FieldService,3 - RegionalFieldService,Amethyst Suki Norris,2009-12
2,Kathleen Audrey Vargas,KAV0428,Kathleen.Audrey.Vargas@dtaa.com,ProductionLineWorker,1,3 - Manufacturing,3 - Assembly,2 - AssemblyDept,Brenna Martha Russell,2009-12
3,Adrienne Joelle Mclean,AJM0772,Adrienne.Joelle.Mclean@dtaa.com,Scientist,1,2 - ResearchAndEngineering,1 - Research,5 - Lab,Emmanuel Casey Wiggins,2009-12
4,Nicole Maris Valentine,NMV0507,Nicole.Maris.Valentine@dtaa.com,Manager,1,5 - SalesAndMarketing,2 - Sales,3 - RegionalSales,Perry Reese Morton,2009-12


In [4]:
# generate users from the compiled 
df_users = pd.DataFrame(df_ldap['user_id'].unique())

In [5]:
# generate known range of date from the window of logs - create string to trunc to other strings
df_dates = pd.date_range('1/1/2010', '5/31/2011').to_frame()
df_dates[0] = df_dates[0].dt.strftime('%Y%m%d')

In [6]:
#initial generation of master dataframe
df_master = pd.merge(df_users.assign(key=1), df_dates.assign(key=1), on='key').drop('key', axis=1)

In [7]:
# check - 516000 rows
df_master

,0_x,0_y
0,MCN0973,20100101
1,MCN0973,20100102
2,MCN0973,20100103
3,MCN0973,20100104
4,MCN0973,20100105
...,...,...
515995,FPS0900,20110527
515996,FPS0900,20110528
515997,FPS0900,20110529
515998,FPS0900,20110530


In [8]:
# rename columns
df_master.rename(columns={'0_y':'dateString','0_x':'user_id'}, inplace=True)

In [9]:
# trunc strings together
df_master['userDate'] = df_master.apply(lambda x: x['user_id'] + '_' + x['dateString'], axis = 1)

In [10]:
# find ending months for each user_id
endDates = df_ldap.groupby('user_id')['date'].max()

In [11]:
# create function to deteermine if a user was employed during a particular date
def isEmployed(row):
    user = row['user_id']
    dateString = int(row['dateString'][0:6])
    # It is a little of a hack, but comparing -1 from 201012 201100 is still ok.
    if dateString == 201101:
        minusMonth = 201012
    else:
        minusMonth = dateString - 1
    if int(endDates[row['user_id']].replace('-','')) >= minusMonth:
        return 1
    else:
        return 0 
    

In [12]:
# applied function
df_master['isEmployed'] = df_master.apply(isEmployed, axis=1)

In [13]:
# generate dictionary of known insiders and dates in which the activity was classified as being an insider threat
insiders = {'CSF0929':['20100701','20100702','20100703','20100708','20100709','20100714','20100716'], 
            'CCH0959':['20100802','20100803','20100804','20100805','20100806','20100809',                                                                                     
                        '20100810','20100811','20100812','20100813','20100816','20100817','20100818',
                        '20100819','20100820','20100823','20100824','20100825','20100826','20100827',
                        '20100830','20100831','20100901','20100902','20100903','20100907','20100908',
                        '20100909','20100910','20100913','20100914','20100915','20100916',
                        '20100917','20100920','20100921','20100922','20100923','20100924','20100927',
                        '20100928','20100929','20100930']}

In [14]:
# function to determine if row should be insider threat
def isInsider(row):
    if row['user_id'] in insiders.keys() and row['dateString'] in insiders[row['user_id']]:
        return 1
    else:
        return 0

In [15]:
# apply function
df_master['isInsider'] = df_master.apply(isInsider, axis=1)

In [16]:
# check
df_master.columns

Index(['user_id', 'dateString', 'userDate', 'isEmployed', 'isInsider'], dtype='object')

In [17]:
# pull in hasLogins pick and apply as hasLogin to row items through merge on userDat
df_hasLogin = pd.read_pickle('../logon/hasLogins.pkl')

df_master = pd.merge(df_master, df_hasLogin, how='left', on='userDate')

df_master.rename(columns={'count':'hasLogin'}, inplace=True)

# checks - logins do not occur if user is not employed
#df_master[(df_master['isEmployed'] == 0) & (df_master['hasLogin'] > 0)]['user_id'].unique()
#print(endDates['HSS0018'])
#print(df_master[(df_master['user_id'] == 'HSS0018') & (df_master['dateString'].str.contains('201101'))])

In [27]:
# check
df_master

,user_id,dateString,userDate,isEmployed,isInsider,hasLogin,firstActivity,hasConnect,role,minHour,probMinLogonRole,maxHour,probMaxLogoutRole,webCount
0,MCN0973,20100101,MCN0973_20100101,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,0.000000,NaN,0.000000,NaN
1,MCN0973,20100102,MCN0973_20100102,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,0.000000,NaN,0.000000,NaN
2,MCN0973,20100103,MCN0973_20100103,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,0.000000,NaN,0.000000,NaN
3,MCN0973,20100104,MCN0973_20100104,1,0,1.0,NaN,NaN,ElectricalEngineer,9.0,0.226949,17.0,0.208013,34.0
4,MCN0973,20100105,MCN0973_20100105,1,0,1.0,NaN,NaN,ElectricalEngineer,9.0,0.226949,17.0,0.208013,219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515995,FPS0900,20110527,FPS0900_20110527,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,0.000000,NaN,0.000000,NaN
515996,FPS0900,20110528,FPS0900_20110528,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,0.000000,NaN,0.000000,NaN
515997,FPS0900,20110529,FPS0900_20110529,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,0.000000,NaN,0.000000,NaN
515998,FPS0900,20110530,FPS0900_20110530,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,0.000000,NaN,0.000000,NaN


In [6]:
# check - num dates
len(df_master['dateString'].unique())

516

In [19]:
# pull in firstActivity from pickle of device and merge on userDate - drop unneeded activity column since filter is on Connect
df_firstActivity = pd.read_pickle('../device/firstActivity.pkl')

df_master = pd.merge(df_master, df_firstActivity[df_firstActivity['activity'] == 'Connect'], how='left', on='userDate')

df_master.drop('activity', axis = 1, inplace=True)

In [20]:
# pull in hasConnect from pickle of device and merge on userDate
df_hasConnect = pd.read_pickle('../device/hasConnect.pkl')

df_master = pd.merge(df_master, df_hasConnect, how='left', on='userDate')

In [21]:
# pull in roles from ldap data set - no changes in role through timeperiod, merge with user_id
df_userRole = pd.read_pickle('../ldap/userRole.pkl')

df_master = pd.merge(df_master, df_userRole, how='left', on='user_id')

In [22]:
# pull in first login for the row item and merge to userDate
# Fixed - this will create duplicate userDate columns - multiple logins per day
df_logonHour = pd.read_pickle('../logon/logonHour.pkl')
df_logonHour = df_logonHour.groupby('userDate')['dateHour'].min().to_frame().reset_index()
df_master = pd.merge(df_master, df_logonHour, how='left', on='userDate').rename(columns={'dateHour':'minHour'})

In [23]:
# pull in probability table for logins based on role as dataframe
# if there is a login for the row item, find the probability and record to probMinLogonRole
df_roleProbHour = pd.read_pickle('../logon/roleProbHour.pkl')

def probHour(row):
    if row['hasLogin'] > 0:
        roleProbMinHour = df_roleProbHour.loc[row['role'], row['minHour']]
    else:
        roleProbMinHour = 0
    
    return roleProbMinHour


df_master['probMinLogonRole'] = df_master.apply(probHour, axis=1)

In [24]:
# pull in the last logout for the row item and merge to userdate
df_logoutHour = pd.read_pickle('../logon/logoutHour.pkl')
df_logoutHour = df_logoutHour.groupby('userDate')['dateHour'].max().to_frame().reset_index()
df_master = pd.merge(df_master, df_logoutHour, how='left', on='userDate').rename(columns={'dateHour':'maxHour'})

In [25]:
# pull in probability table for logouts based on role as dataframe
# if there is a login for the day, find the probability and report for probMaxLogoutRole
df_roleProbLogoutHour = pd.read_pickle('../logon/roleProbLogoutHour.pkl')

def probHour(row):
    if row['hasLogin'] > 0:
        roleProbMaxHour = df_roleProbLogoutHour.loc[row['role'], row['maxHour']]
    else:
        roleProbMaxHour = 0
    
    return roleProbMaxHour


df_master['probMaxLogoutRole'] = df_master.apply(probHour, axis=1)


In [26]:
# read in the url by user date pickle and merge..
df_urlByuserDate = pd.read_pickle('../http/urlByUserDate.pkl')
df_master = pd.merge(df_master, df_urlByuserDate.rename(columns={'count':'webCount'}), how='left', on='userDate')

# test for weirdness : df_temp[(df_temp['hasLogin'] == 0) & (df_temp['webCount'] > 0)]
# df_temp[(df_temp['hasLogin'] > 0) & (df_temp['webCount'] > 0)]
# df_temp[(df_temp['hasLogin'] > 0) & (df_temp['webCount'] > 0)]


20230304 - added questionable usage

In [28]:
# read in the question usage pickle and merge
df_webQuestionsableUsage = pd.read_pickle('../http/insiderWebCount.pkl')
df_master = pd.merge(df_master, df_webQuestionsableUsage.rename(columns={'isInsiderWeb':'webQuestionableUsage'}), how='left', on='userDate')

# more tests for weirdness
# df_master[(df_master['hasLogin'] > 0) & (df_master['webCount'] > 0)]

20230224 - added numPC and psychometric

In [29]:
# read in the number of distinct PC logins for row item
df_numPC = pd.read_pickle('../logon/numPC.pkl').reset_index()
df_master = pd.merge(df_master, df_numPC, how='left', on='userDate')


#df_test[(df_test['countPC'] > 1) & ~(np.isnan(df_test['hasLogin']))] 
# may dup hasLogin

In [30]:
# read in OCEAN data for users. static for user_id. merged to user_id
df_psychometic = pd.read_pickle('../psychometric/psychometic.pkl')
df_master = pd.merge(df_master, df_psychometic, how='left', on='user_id')

In [31]:
# read in number of months employed pickle. merge and rename columns
df_numMonthsEmployed = pd.read_pickle('../ldap/numMonthsEmployed.pkl')
df_numMonthsEmployed = df_numMonthsEmployed.to_frame().reset_index().rename(columns={'date':'numMonthsEmployed'})

#check
#df_numMonthsEmployed[df_numMonthsEmployed['numMonthsEmployed'] != 18]
df_master = pd.merge(df_master, df_numMonthsEmployed, how='left', on='user_id')

In [32]:
# read in email metrics for row items. merge on userDate
df_emailCounts = pd.read_pickle('../email/emailCounts.pkl')
df_master = pd.merge(df_master, df_emailCounts, how='left', on='userDate')

In [33]:
# using same minHour. record probability for user rather than role
df_userProbHour = pd.read_pickle('../logon/userProbHour.pkl')

def probUser(row):
    if row['hasLogin'] > 0:
        probLogonUser = df_userProbHour.loc[row['user_id'], row['minHour']]
    else:
        probLogonUser = 0
    
    return probLogonUser

df_master['probLogonUser'] = df_master.apply(probUser, axis=1)

In [34]:
# using same maxHour. record probability for user rather than role
df_userProbLogoutHour = pd.read_pickle('../logon/userProbLogoutHour.pkl')

def probUser(row):
    if row['hasLogin'] > 0:
        probLogoffUser = df_userProbLogoutHour.loc[row['user_id'], row['maxHour']]
    else:
        probLogoffUser = 0
    
    return probLogoffUser

df_master['probLogoffUser'] = df_master.apply(probUser, axis=1)

In [35]:
# check for change in roles for user during full time period
df_changeRoles = pd.read_pickle('../ldap/changeRoles.pkl')
df_changeRoles = df_changeRoles.set_index('dateString')

def changeInRoles(row):
    changeInRole = df_changeRoles.loc[row['dateString'],row['role']]
    return changeInRole

df_master['changeNumRole'] = df_master.apply(changeInRoles, axis=1)
# test : df_master[(df_master['changeNumRole'] > 0 ) & (df_master['role'] == "AdministrativeAssistant")]

In [ ]:
#not longer needed - note on num logins
#df_master.drop_duplicates(inplace=True)

In [40]:
# write to parquet file
df_master.to_parquet('./master.parquet')

In [39]:
# sample row basic items
df_master[['user_id', 'dateString', 'userDate','role','isInsider','employee_name']].iloc[0]

user_id                     MCN0973
dateString                 20100101
userDate           MCN0973_20100101
role             ElectricalEngineer
isInsider                         0
employee_name    Macey Colleen Nash
Name: 0, dtype: object

In [42]:
# check non-insider threats
df_master[df_master['isInsider'] == 0]

,user_id,dateString,userDate,isEmployed,isInsider,hasLogin,firstActivity,hasConnect,role,minHour,...,psyc_E,psyc_A,psyc_N,numMonthsEmployed,sumExtEmailSize,sumExtEmailAttachments,numExtEmails,probLogonUser,probLogoffUser,changeNumRole
0,MCN0973,20100101,MCN0973_20100101,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,...,41,14,29,18,NaN,NaN,NaN,0.000000,0.000000,0
1,MCN0973,20100102,MCN0973_20100102,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,...,41,14,29,18,NaN,NaN,NaN,0.000000,0.000000,0
2,MCN0973,20100103,MCN0973_20100103,1,0,NaN,NaN,NaN,ElectricalEngineer,NaN,...,41,14,29,18,NaN,NaN,NaN,0.000000,0.000000,0
3,MCN0973,20100104,MCN0973_20100104,1,0,1.0,NaN,NaN,ElectricalEngineer,9.0,...,41,14,29,18,21124.0,0.0,1.0,0.398942,0.398942,0
4,MCN0973,20100105,MCN0973_20100105,1,0,1.0,NaN,NaN,ElectricalEngineer,9.0,...,41,14,29,18,NaN,NaN,NaN,0.398942,0.398942,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515995,FPS0900,20110527,FPS0900_20110527,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,39,16,30,18,NaN,NaN,NaN,0.000000,0.000000,1
515996,FPS0900,20110528,FPS0900_20110528,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,39,16,30,18,NaN,NaN,NaN,0.000000,0.000000,1
515997,FPS0900,20110529,FPS0900_20110529,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,39,16,30,18,NaN,NaN,NaN,0.000000,0.000000,1
515998,FPS0900,20110530,FPS0900_20110530,1,0,NaN,NaN,NaN,ComputerProgrammer,NaN,...,39,16,30,18,NaN,NaN,NaN,0.000000,0.000000,1


In [ ]:
# no longer needed switch to parquet
#df_master.to_pickle('./master.pkl')

In [ ]:
# no longer needed switch to parquet
#df_master = pd.read_pickle('./master.pkl')

In [ ]:
# check
df_master[['dateString', 'userDate']]